In [1]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet


import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,lpad, length, lit, to_date,to_timestamp, when, date_format, coalesce, md5, sha2, concat_ws,trim

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Qlik_transfo") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 11:37:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path_ = "/Users/dataworkshop/Desktop/Spark/datasetV1.xlsx"

file_path = "/Users/dataworkshop/Desktop/Spark/Frasi_Gestion.xlsx"
Suivi_et_Archive = "Suivi_et_Archive"
Tab_PARAM = "Tab_PARAM"
PARAM_FG = "PARAM_FG"

In [4]:
df_Suivi_et_Archive = pd.read_excel(file_path_, header=1)
df_Tab_PARAM = pd.read_excel(file_path, Tab_PARAM)
df_PARAM_FG = pd.read_excel(file_path, PARAM_FG)

Param FG

In [5]:
# effecaer les colonnes qui sont vides
columns_to_delete = ['Unnamed: 9', 'Unnamed: 12',"Unnamed: 16",]
# Effacer les colonnes vide
df_PARAM_FG = df_PARAM_FG.drop(columns=columns_to_delete)


In [6]:
schema_PARAM_FG = StructType([
    StructField('Unnamed: 0', StringType(), True),
    StructField('Année', StringType(), True),
    StructField('Catégorie', StringType(), True),
    StructField('Sous-Catégorie',StringType(), True),
    StructField('Recueillir les candidatures TTC', IntegerType(), True),
    StructField('Instruire les dossiers TTC',IntegerType(), True),
    StructField('Accompagner les projets TTC', IntegerType(), True),
    StructField('Recueillir les candidatures HT',IntegerType(), True),
    StructField('Instruire les dossiers HT', IntegerType(), True),
    StructField('Accompagner les projets HT',IntegerType(), True),
    StructField('Reporting et évaluation TTC', IntegerType(), True),
    StructField('Reporting et évaluation HT',IntegerType(), True),
    StructField('Convention SCAN', StringType(), True),
    StructField('Convention', StringType(), True),
    StructField('Sous-fonds FG AAP/Projets',StringType(), True),
    StructField('Dispositif', StringType(), True),
    StructField('Dispositif Cible', StringType(), True),
    StructField('AAP', StringType(), True),
    StructField('Date de relève',StringType(), True),
    StructField('Acronyme du projet', StringType(), True),
    StructField("Nom de l'entreprise", StringType(), True),
    StructField("Identifiant projet",StringType(), True),
    StructField("Montant total de l'aide retenue HT", IntegerType(), True),
    StructField('COPIL : Date de signature DPM',StringType(), True),
    StructField('Convention.1', StringType(), True),
    StructField('AAP_Gré-à-gré', StringType(), True),])



# MAPPING
PARAM_FG = spark.read.csv('df_PARAM_FG_clean.csv', header=True, schema=schema_PARAM_FG)

# Replace NaN values with None
PARAM_FG = PARAM_FG.na.replace(float('nan'), None)

In [7]:
PARAM_FG.show(truncate=False)  

23/12/21 11:37:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/12/21 11:37:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Année, Catégorie, Sous-Catégorie, Recueillir les candidatures TTC, Instruire les dossiers TTC, Accompagner les projets TTC, Recueillir les candidatures HT, Instruire les dossiers HT, Accompagner les projets HT, Reporting et évaluation TTC, Reporting et évaluation HT, Convention SCAN, Convention, Sous-fonds FG AAP/Projets, Dispositif, Dispositif Cible, AAP, Date de relève, Acronyme du projet, Nom de l'entreprise, Identifiant projet, Montant total de l'aide retenue HT, COPIL : Date de signature DPM, Convention.1, AAP_Gré-à-gré
 Schema: Unnamed: 0, Année, Catégorie, Sous-Catégorie, Recueillir les candidatures TTC, Instruire les dossiers TTC, Accompagner les projets TTC, Recueillir les candidatures HT, Instruir

+----------+------+---------------+--------------+-------------------------------+--------------------------+---------------------------+------------------------------+-------------------------+--------------------------+---------------------------+--------------------------+----------------------+--------------------+-------------------------+----------+----------------+--------------+--------------+------------------------------------------------+------------------------------------------+---------------------------------------+----------------------------------+-----------------------------+----------------------+------------------------------------+
|Unnamed: 0|Année |Catégorie      |Sous-Catégorie|Recueillir les candidatures TTC|Instruire les dossiers TTC|Accompagner les projets TTC|Recueillir les candidatures HT|Instruire les dossiers HT|Accompagner les projets HT|Reporting et évaluation TTC|Reporting et évaluation HT|Convention SCAN       |Convention          |Sous-fonds FG AAP/

In [8]:
df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))
df.show()

/var/folders/6d/b6p57_cn0t9g6b9c6t40smg80000gn/T/ipykernel_88779/2821287331.py:1: DtypeWarning: Columns (77,78,79,80,109,112,121,122,123,124,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
23/12/21 11:37:30 WARN TaskSetManager: Stage 1 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+-------------------+-------------------+--------------------+--------------------+------------+---------+--------------------+-----------------+----------+--------------------+---------------------------------------+---------------------------------+-----------------------------------+--------------------+---------------------+-----------------------------+------------------+------------+---------+-----------------------------------+--------------------------------------+----------------------------------------+---------------------------------------------+------------------------------------+---------------------------------------+-----------------------------------------+----------------------------------------------+--------------------+---------------------------+---------------------------------------+---------------+------------------------------+---------------------------+---------------------------+-------------+-----------------------+------------------

23/12/21 11:37:34 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


In [9]:
Param = df_Tab_PARAM[['Statut', "Terminé/En cours",'Liste_Relèves',"Stade_Relèves_FG"]]

Param = spark.createDataFrame(Param)
# rename stat to be able to map terminé/en cours properly
Param = Param.withColumnRenamed("Statut", "Statut_")

Param.show()

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+--------------------+----------------+--------------------+--------------------+
|             Statut_|Terminé/En cours|       Liste_Relèves|    Stade_Relèves_FG|
+--------------------+----------------+--------------------+--------------------+
|Analyse opportuni...|        Terminés|     2021 -  févr-21|Majoritairement t...|
|Attente dépôt com...|        En cours|     2022 -  févr-22|Majoritairement t...|
| Audition: Abandonné|        Terminés|     2023 -  févr-23|Majoritairement t...|
|   Audition: Ajourné|        En cours|ACCEL CYBER AMI 1...|Majoritairement t...|
|Audition: Défavor...|        Terminés|AERONEF BAS CARBO...|Majoritairement t...|
|Audition: Réorien...|        En cours|AERONEF BAS CARBO...|Majoritairement t...|
|     COEX: Abandonné|        Terminés|AGROALIM 1 -  jan...|Majoritairement t...|
|       COEX: Ajourné|        En cours|AGROALIM 1 -  jui...|Majoritairement t...|
|   COEX: Défavorable|        Terminés|AGROALIM 1 -  mar...|Majoritairement t...|
|     COEX: Favo

lire fichier param_cadrage

In [10]:
# Specify the path to your Excel file
excel_file_path = "/Users/dataworkshop/Desktop/Spark/FG_PARAM.xlsx"

# Read the Excel file into a Pandas DataFrame
FG_Param = pd.read_excel(excel_file_path)

# Convert the Pandas DataFrame to a PySpark DataFrame
FG_Param = spark.createDataFrame(FG_Param)

FG_Param = FG_Param.withColumnRenamed("AAP", "_AAP")


# Show the content of the PySpark DataFrame
FG_Param.show()

+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|                _AAP|Catégorie_AAP|      Convention_AAP|AAP Actifs 2021|AAP Actifs 2022|AAP Actifs 2023|AAP Actifs 2024|AAP Actifs 2025|AAP Actifs 2026|AAP Actifs 2027|AAP Actifs 2028|AAP Actifs 2029|AAP Actifs 2030|AAP Actifs 2031|AAP Actifs 2032|AAP Actifs 2033|AAP Actifs 2034|AAP Actifs 2035|
+--------------------+-------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|AERONEF BAS CARBONE1|    SA/FR2030|Démonstrateur Dirigé|              0|              1|              1| 

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


# F_G_projets_Phase_1

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import year
from pyspark.sql.functions import col, sum, min, max
from pyspark.sql.functions import first




F_G_projet_1 = df[["Identifiant projet", 'Dispositif / Opérateur cible',"AAP", 'Acronyme du projet', 'Instruction affectée autre opérateur', 
                 'COPIL : Date de signature DPM',"Montant total de l'aide retenue HT", 'Date de verrouillage',"COPIL : Décision Comité"]]

# str to int

F_G_projet_1 = F_G_projet_1.withColumn("Montant total de l'aide retenue HT", col("Montant total de l'aide retenue HT").cast("int"))


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé 'Dispositif / Opérateur cible' ==> Dispositif de sortie
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Dispositif / Opérateur cible", "Dispositif de sortie")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Catégorie Dispositif  ==> Convention AAP dans F_G_AAP_Phase_Cadrage

FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param1, F_G_projet_1["AAP"] == FG_Param1["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("_AAP")

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Convention AAP ==> convention dans F_G_AAP_Phase_Cadrage
FG_Param2 =FG_Param[["_AAP","Catégorie_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param2, F_G_projet_1["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_1 = F_G_projet_1.drop("_AAP")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# sous-fonds FG == 

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé Instruction affectée autre opérateur ==> Instruction autre opérateur
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Instruction affectée autre opérateur", "Instruction autre opérateur")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projet Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "

condition_nb_projet_decide = col("COPIL : Décision Comité") == 'Favorable'
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projet décidés COPIL", when(condition_nb_projet_decide, 1).otherwise(None))
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de partenaire Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "
# cette colonne sera créer dans le groupby

# Assuming F_G_projet_1 is your DataFrame
F_G_Nombre_patrtenaire = F_G_projet_1.groupBy("Identifiant projet").agg(
    F.sum("Nombre de projet décidés COPIL").alias("Nombre de partenaire décidés COPIL")
)
F_G_Nombre_patrtenaire = F_G_Nombre_patrtenaire.withColumnRenamed("Identifiant projet", "Identifiant projet_")

F_G_projet_1 = F_G_projet_1.join(F_G_Nombre_patrtenaire, F_G_projet_1["Identifiant projet"] == F_G_Nombre_patrtenaire["Identifiant projet_"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("Identifiant projet_")


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projets verrouillés : Sum of notnull values in date de verrouillage
confition_nb_pr_verrouillés = col("Date de verrouillage").isNotNull()
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projets verrouilles", when(confition_nb_pr_verrouillés, 1).otherwise(0))

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Mapper Reveuillir les candidatures deuis Param_FG

receuille_candi =PARAM_FG[["Année","Catégorie","Recueillir les candidatures HT"]]

#format année
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(IntegerType()))
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(StringType()))

# Receuillir_key
receuille_candi = receuille_candi.withColumn("Receuillir_key", concat_ws("_", col("Année"), col("Catégorie"))) 
#receuille_candi = receuille_candi.drop("Année")
#receuille_candi = receuille_candi.drop("Catégorie")


# create year 
F_G_projet_1 = F_G_projet_1.withColumn("Date de verrouillage", F.to_date(col("Date de verrouillage")))
F_G_projet_1 = F_G_projet_1.withColumn("Année de verrouillage" , year(F_G_projet_1["Date de verrouillage"]))

F_G_projet_1 = F_G_projet_1.withColumn("Receuillir_key_", concat_ws("_", col("Année de verrouillage"), col("Catégorie Dispositif"))) 


F_G_projet_1 = F_G_projet_1.join(receuille_candi, F_G_projet_1["Receuillir_key_"] == receuille_candi["Receuillir_key"], "left_outer") 

F_G_projet_1 = F_G_projet_1.drop("Receuillir_key_")
F_G_projet_1 = F_G_projet_1.drop("Receuillir_key")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------

# Display the similar version of 

F_G_projet_1_ = F_G_projet_1.groupBy("Identifiant projet").agg(
    first("Dispositif de sortie").alias("Dispositif de sortie"),
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention_AAP").alias("Convention AAP"),
    first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Date de verrouillage").alias("Date de verrouillage"),    
    first("Nombre de projets verrouilles").alias("Nombre de projets verrouilles"),
    max("Recueillir les candidatures HT").alias("Recueillir les candidatures HT"),
)

# Show the result
F_G_projet_1.show(truncate=True)

23/12/21 11:37:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/12/21 11:37:35 WARN TaskSetManager: Stage 4 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:36 WARN TaskSetManager: Stage 7 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+------------------+---------------------------+-----------------------------+----------------------------------+--------------------+-----------------------+--------------------+--------------------+------------------------------+----------------------------------+-----------------------------+---------------------+---------------+-----+---------+------------------------------+--------------+
|  Identifiant projet|Dispositif de sortie|                 AAP|Acronyme du projet|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Date de verrouillage|COPIL : Décision Comité|      Convention_AAP|Catégorie Dispositif|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Nombre de projets verrouilles|Année de verrouillage|Receuillir_key_|Année|Catégorie|Recueillir les candidatures HT|Receuillir_key|
+--------------------+--------------------+--------------------+----------------

In [12]:
F_G_projet_1_.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

23/12/21 11:37:37 WARN TaskSetManager: Stage 26 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:37 WARN TaskSetManager: Stage 29 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+--------------------+-----------+------------------+--------------------+--------------+---------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+--------------------+-----------------------------+------------------------------+
|Identifiant projet        |Dispositif de sortie|AAP        |Acronyme du projet|Catégorie Dispositif|Convention AAP|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Date de verrouillage|Nombre de projets verrouilles|Recueillir les candidatures HT|
+--------------------------+--------------------+-----------+------------------+--------------------+--------------+---------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+---------

In [13]:
F_G_projet_1.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

23/12/21 11:37:38 WARN TaskSetManager: Stage 50 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:38 WARN TaskSetManager: Stage 53 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+--------------------+-----------+------------------+---------------------------+-----------------------------+----------------------------------+--------------------+-----------------------+--------------+--------------------+------------------------------+----------------------------------+-----------------------------+---------------------+---------------+-----+---------+------------------------------+--------------+
|Identifiant projet        |Dispositif de sortie|AAP        |Acronyme du projet|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Date de verrouillage|COPIL : Décision Comité|Convention_AAP|Catégorie Dispositif|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Nombre de projets verrouilles|Année de verrouillage|Receuillir_key_|Année|Catégorie|Recueillir les candidatures HT|Receuillir_key|
+--------------------------+--------------------+-----------+------------------+------------------

# F_G_Projet_Phase_2_3

Old code 

In [14]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import translate


# create month & year to transform liste_Relève
df = df.withColumn("Month de relève", date_format("Date de relève", "MMM"))
df = df.withColumn("Year de relève", date_format("Date de relève", "yy"))



# Define your conditions
jan = col("Month de relève") == "Jan"
fev = col("Month de relève") == "Feb"
mar = col("Month de relève") == "Mar"
apr = col("Month de relève") == "Apr"
mai = col("Month de relève") == "May"
jun = col("Month de relève") == "Jun"
july = col("Month de relève") == "Jul"
aug = col("Month de relève") == "Aug"
sep = col("Month de relève") == "Sep"
oct = col("Month de relève") == "Oct"
nov = col("Month de relève") == "Nov"
dec = col("Month de relève") == "Dec"

# Apply the conditions and create Résultat de l'instruction Autres/AMI
df = df.withColumn(
    "Month de relève",
    when(jan, "janv")
    .when(fev, "févr")
    .when(mar, "mars")
    .when(apr, "avr")
    .when(mai, "Mai")
    .when(jun, "juin")
    .when(july, "juil")
    .when(aug, "août")
    .when(sep, "sept")
    .when(oct, "oct")
    .when(nov, "nov")
    .when(dec, "déc")
    .otherwise(col("Month de relève"))
)

# Concatener mois_année

df = df.withColumn("month/year", concat_ws("-", col("Month de relève"), col("Year de relève"))) 
df = df.withColumn("Liste_Relèves_", concat_ws(" - ", col("AAP"), col("month/year")))

In [15]:
F_G_projet_2_3 = df[["Identifiant projet", 'Dispositif', "AAP", 'Acronyme du projet','Statut',
                   'Convention','Instruction affectée autre opérateur','COPIL : Date de signature DPM',
                   'COEX : Date de validation du PV de présélection', 'Date de signature client',
                   "COEX : Décision Comité","Date de relève",'COPIL : Décision Comité',"Montant total de l'aide retenue HT"]]
#________________---------------------------

# Convention AAP ==> convention dans F_G_AAP_Phase_Cadrage
FG_Param2 =FG_Param[["_AAP","Catégorie_AAP"]]
F_G_projet_2_3 = F_G_projet_2_3.join(FG_Param2, F_G_projet_2_3["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_2_3 = F_G_projet_2_3.drop("_AAP")



#--1-- Mapper stade projet depuis tab_param & rename it to stade projet

F_G_projet_2_3 = F_G_projet_2_3.join(Param, F_G_projet_2_3["Statut"] == Param["Statut_"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Statut_")
# Rename Terminé/En cours to Stade
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Terminé/En cours", "Stade projet")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--2-- créer stade relève à partir de regle de gestion dans neurone


# create month & year to transform liste_Relève
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Month de relève", date_format("Date de relève", "MMM"))
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Year de relève", date_format("Date de relève", "yy"))



# Define your conditions
jan = col("Month de relève") == "Jan"
fev = col("Month de relève") == "Feb"
mar = col("Month de relève") == "Mar"
apr = col("Month de relève") == "Apr"
mai = col("Month de relève") == "May"
jun = col("Month de relève") == "Jun"
july = col("Month de relève") == "Jul"
aug = col("Month de relève") == "Aug"
sep = col("Month de relève") == "Sep"
oct = col("Month de relève") == "Oct"
nov = col("Month de relève") == "Nov"
dec = col("Month de relève") == "Dec"

# Apply the conditions and create Résultat de l'instruction Autres/AMI
F_G_projet_2_3 = F_G_projet_2_3.withColumn(
    "Month de relève",
    when(jan, "janv")
    .when(fev, "févr")
    .when(mar, "mars")
    .when(apr, "avr")
    .when(mai, "Mai")
    .when(jun, "juin")
    .when(july, "juil")
    .when(aug, "août")
    .when(sep, "sept")
    .when(oct, "oct")
    .when(nov, "nov")
    .when(dec, "déc")
    .otherwise(col("Month de relève"))
)

# Concatener mois_année
F_G_projet_2_3 = F_G_projet_2_3.withColumn("month/year", concat_ws("-", col("Month de relève"), col("Year de relève"))) # trim vID_Partenaire
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Liste_Relèves_", concat_ws(" - ", col("AAP"), col("month/year"))) # trim vID_Partenaire


# get the values to create Stade_relève
Param_stade_relève = Param[['Liste_Relèves',"Stade_Relèves_FG"]]
Param_stade_relève = Param_stade_relève.withColumnRenamed("Stade_Relèves_FG", "Stade_Relèves_FG_")

#jointure Stade_relève
F_G_projet_2_3 = F_G_projet_2_3.join(Param_stade_relève, F_G_projet_2_3["Liste_Relèves_"] == Param_stade_relève["Liste_Relèves"], "left")
F_G_projet_2_3 = F_G_projet_2_3.drop("Liste_Relèves")
F_G_projet_2_3 = F_G_projet_2_3.drop("Liste_Relèves_")

F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Stade_Relèves_FG", "Stade Relèves")
F_G_projet_2_3 = F_G_projet_2_3.drop("Stade_Relèves_FG_")


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--3-- renommer convention à convention projet SCAN
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("convention", "Convention Projet SCAN")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--4-- Mapper Convention projet pour FG ( Mapper la colonne de Param_FG )
Conv_Map = PARAM_FG[["Convention SCAN", "Convention"]]
#jointure convention
F_G_projet_2_3 = F_G_projet_2_3.join(Conv_Map, F_G_projet_2_3["Convention Projet SCAN"] == Conv_Map["Convention SCAN"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Convention SCAN")
# rename Convention
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("convention", "Convention Projet pour FG")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#--5-- ramener Nombre de projet Décidés COPIL
ondition_nb_projet_decide = col("COPIL : Décision Comité") == 'Favorable'
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de projet décidés COPIL", when(ondition_nb_projet_decide, 1).otherwise(None))
#-------------------------------------------------------------------------------------------------------------------------------------------------------
#--6-- ramener Nombre de partenaire Décidés COPIL


F_G_Nombre_patrt = F_G_projet_2_3.groupBy("Identifiant projet").agg(
    F.sum("Nombre de projet décidés COPIL").alias("Nombre de partenaire décidés COPIL")
    )
F_G_Nombre_patrt = F_G_Nombre_patrt.withColumnRenamed("Identifiant projet", "Identifiant projet_")

F_G_projet_2_3 = F_G_projet_2_3.join(F_G_Nombre_patrt, F_G_projet_2_3["Identifiant projet"] == F_G_Nombre_patrt["Identifiant projet_"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Identifiant projet_")



#--7-- créer Projet Validé COEX ( RG dans dans neurone)

confition_proj_vali_coex = col("COEX : Décision Comité") == "Favorable"
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Projet Validé COEX", when(confition_proj_vali_coex, "Oui").otherwise("Non"))


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer nombre de Projet Validé COEX ( RG dans dans neurone)

confition_nb_proj_vali_coex = col("COEX : Décision Comité") == "Favorable"
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de Projet Validé COEX", when(confition_nb_proj_vali_coex, 1).otherwise(0))

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# renommer 'COEX : Date de validation du PV de présélection à Date de COEX ( RG )

F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("COEX : Date de validation du PV de présélection", "Date de COEX")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Mapper Instruire les dossiers à partir de PARAM_FG

instruire =PARAM_FG[["Catégorie","Instruire les dossiers HT"]]
F_G_projet_2_3 = F_G_projet_2_3.join(instruire, F_G_projet_2_3["Catégorie Dispositif"] == instruire["Catégorie"], "left_outer") 


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# renommer  'Date de signature client' à Date de contractualisation
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Date de signature client", "Date de contractulisation")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer date de fin de projet ( à voir avec le métier )
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer Nombre de projets contractualisés ( RG dans neurone)
confition_nb_proj_cont = col("Date de contractulisation").isNotNull()
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de projets contractulisation", when(confition_nb_proj_cont, 1).otherwise(0))
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Mapper Accompagner les projets - Annéé 1 jusqu'a 10

accompagner =PARAM_FG[["Année","Catégorie","Accompagner les projets HT"]]


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Accompagner les projets -total ( RG dans neurone)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer col Frais de gestion suivi par année ==> moyenne de toutes les colonnes Accompagner les projets
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer Nb Années de suivis ( RG dans neurone)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer accompagner les pojets -2021 jusqu'a pojets -2030 
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Créer Accompagner les projets -total
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


F_G_projet_2_3 = F_G_projet_2_3.drop("Statut")

F_G_projet_2_3.show(truncate=False)

23/12/21 11:37:39 WARN TaskSetManager: Stage 74 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:40 WARN TaskSetManager: Stage 78 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------------------+----------+-----------------+-------------------+----------------------+------------------------------------+-----------------------------+-------------------+-------------------------+----------------------+-------------------+-----------------------+----------------------------------+--------------------+------------+-----------------------+---------------+--------------+----------+-------------------------+------------------------------+----------------------------------+------------------+----------------------------+---------+-------------------------+-----------------------------------+
|Identifiant projet                   |Dispositif|AAP              |Acronyme du projet |Convention Projet SCAN|Instruction affectée autre opérateur|COPIL : Date de signature DPM|Date de COEX       |Date de contractulisation|COEX : Décision Comité|Date de relève     |COPIL : Décision Comité|Montant total de l'aide retenue HT|Catégorie Dispositif|Stade projet|S

In [16]:
F_G_projet_2_3_ = F_G_projet_2_3
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Montant total de l'aide retenue HT", col("Montant total de l'aide retenue HT").cast("int"))
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Instruire les dossiers HT", col("Instruire les dossiers HT").cast("int"))
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Nombre de projets contractulisation", col("Nombre de projets contractulisation").cast("int"))


# Display the similar version of 

F_G_projet_2_3_ = F_G_projet_2_3.groupBy("Identifiant projet").agg(
    first("Dispositif").alias("Dispositif"),
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Stade projet").alias("Stade projet"),
    first("Stade Relèves").alias("Stade Relèves"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention Projet SCAN").alias("Convention Projet SCAN"),
    first("Convention Projet pour FG").alias("Convention Projet pour FG"),
    #first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Projet Validé COEX").alias("Projet Validé COEX"),
    first("Nombre de Projet Validé COEX").alias("Nombre de Projet Validé COEX"),  
    first("Date de COEX").alias("Date de COEX"),
    max("Instruire les dossiers HT").alias("Instruire les dossiers HT"),
    max("Date de contractulisation").alias("Date de contractulisation"),
    max("Nombre de projets contractulisation").alias("Nombre de projets contractulisation"),
)

In [17]:
F_G_projet_2_3.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

23/12/21 11:37:41 WARN TaskSetManager: Stage 104 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:41 WARN TaskSetManager: Stage 108 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+----------+-----------+------------------+----------------------+------------------------------------+-----------------------------+-------------------+-------------------------+----------------------+-------------------+-----------------------+----------------------------------+--------------------+------------+-----------------------+---------------+--------------+----------+-------------------------+------------------------------+----------------------------------+------------------+----------------------------+---------+-------------------------+-----------------------------------+
|Identifiant projet        |Dispositif|AAP        |Acronyme du projet|Convention Projet SCAN|Instruction affectée autre opérateur|COPIL : Date de signature DPM|Date de COEX       |Date de contractulisation|COEX : Décision Comité|Date de relève     |COPIL : Décision Comité|Montant total de l'aide retenue HT|Catégorie Dispositif|Stade projet|Stade Relèves          |Month de relè

In [18]:
F_G_projet_2_3_.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

23/12/21 11:37:42 WARN TaskSetManager: Stage 144 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
23/12/21 11:37:43 WARN TaskSetManager: Stage 148 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+----------+-----------+------------------+------------+-----------------------+--------------------+----------------------+-------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+------------------+----------------------------+-------------------+-------------------------+-------------------------+-----------------------------------+
|Identifiant projet        |Dispositif|AAP        |Acronyme du projet|Stade projet|Stade Relèves          |Catégorie Dispositif|Convention Projet SCAN|Convention Projet pour FG|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Projet Validé COEX|Nombre de Projet Validé COEX|Date de COEX       |Instruire les dossiers HT|Date de contractulisation|Nombre de projets contractulisation|
+--------------------------+----------+-----------+------------------+

# F_G_Global_AAP

In [19]:
F_G_Global_AAP = [["Dispositif de sortie","AAP","Date de relève"]]

# renommer "Dispositif de sortie" à Dispositif
# Renommer AAP ==> Code AAP
# Mapper Procédure
# renommer "Date de relève" ==> 1er année de relève et choisir la date min
# renommer Catégorie Dispositif à Catégorie AAP

#------------------------------------ Frais de gestion --------------------------------------------------------

# Créer Cadrage AAP ( RG dans neurone)
# créer Phase receuil et préselection ( RG dans neurone )
# Phase instruction ( RG dans neurone )
# créer Phase suivi ( RG dans neurone)
# créer phase Suivi 2021
# créer phase Suivi 2022
# créer reporting ( RG dans neurone )

#------------------------------------ Multiplicateurs --------------------------------------------------------

# créer Nb eojets phase receuil et préselection ( RG dans neurone )
# créer Nb projets phase instruction ( RG dans neurone )
# créer Nb projets phase Suivi ( RG dans neurone )
# créer Nb Années ( RG dans neurone )
# créer Nb Experts dans Audition ( RG à confirmer )
# Mapper Nb expertises ( voir quel est la table de param à mapper)
# Nombre de projets décidés ( RG dans neurone )
# créer montant aides décidées ( RG dans neurone )
# créer FG par projet décidés ( RG dans neurone)
# créer % 


F_G_Global_convention = [[]]
# a regarder dans neurone

In [20]:
# Display the similar version of 

F_G_Global_convention = F_G_projet_2_3.groupBy("Dispositif").agg(
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Stade projet").alias("Stade projet"),
    first("Stade Relèves").alias("Stade Relèves"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention Projet SCAN").alias("Convention Projet SCAN"),
    first("Convention Projet pour FG").alias("Convention Projet pour FG"),
    #first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    sum("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Projet Validé COEX").alias("Projet Validé COEX"),
    first("Nombre de Projet Validé COEX").alias("Nombre de Projet Validé COEX"),  
    first("Date de COEX").alias("Date de COEX"),
    max("Instruire les dossiers HT").alias("Instruire les dossiers HT"),
    max("Date de contractulisation").alias("Date de contractulisation"),
)

In [21]:
F_G_projet_last =F_G_projet_2_3[['Dispositif',
 'AAP',
 'Date de relève']]

In [22]:

F_G_projet_last.filter(col("Dispositif")=="30 AUTO").show(100)

23/12/21 11:37:43 WARN TaskSetManager: Stage 184 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+-------------------+
|Dispositif|                 AAP|     Date de relève|
+----------+--------------------+-------------------+
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 2022|2022-06-09 00:00:00|
|   30 AUTO|          CORAM 